In [1]:
import sys,re
sys.path.append('../')
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd 
import numpy as np 

In [125]:
import csv 
outputfolder = "/Users/amycweng/DH/SERMONS_APP/db/data"
new = []
bible = {}
# doc_id,version,part,book,chapter,verse,text
kjv = pd.read_csv('../assets/bible/kjv.csv')
for idx,verse_id in enumerate(kjv['doc_id']): 
    orig_id = verse_id
    verse_id = verse_id.split("(KJV)")[0].strip(" ")
    verse_id = re.sub(r"[\s\:]","-",verse_id)
    bible[verse_id] = kjv['text'][idx]
    new.append({0:orig_id, 
                1:kjv['version'][idx],
                2:kjv['part'][idx],
                3:kjv['book'][idx],
                4:kjv['chapter'][idx],
                5:kjv['verse'][idx],
                6:kjv['text'][idx]})
with open(f"{outputfolder}/kjv.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=new[0].keys())
    writer.writerows(new)

In [139]:
import re

def split_verse(segment):
    pattern = r'\s+([,.?!;:)])'
    segment = re.sub(pattern, r'\1', segment)
    pattern = r'([(])\s+'
    segment = re.sub(pattern, r'\1', segment)
    
    if len(segment.split(" ")) < 32: # below 75th percentile  
        return [segment]
    subsegments = re.split(r"\.|\;|\?|\!",segment)
    # to_segment = ["but", ", while", ", let", ", they", ", NONLATINALPHABET",
    #                 ", then", ", yet", ", than", ', and yet', ', and though',
    #                 ', at least', ', and to', ', this be', ', for', ', therefore',
    #                 ', that', ', and we', ', and i ', ', when', ', and say', ', and this',
    #                 ', and then', ', and than', ', and they', ', i say', ', as the apostle',
    #                 ', otherwise', ', how', ', according', ', accordi^^', ', say',', and when',
    #                 ', and he', ', and she', ', he say', ', she say', ', lest', ', and where',
    #                 ', and how', ', and what', ', and there', ', and therefore', ', and thus',
    #                 ', and if', ', and because', ', and I ', ', he will', ', they will', ', she will']
    # pattern = '|'.join(map(re.escape, to_segment))
    # all_parts = []
    # for segment in subsegments: 
    #     parts = re.split(pattern, segment)
    #     matches = re.findall(pattern,segment)
    #     for idx, part in enumerate(parts):
    #         if idx == (len(parts) - 1): break
    #         # if len(part) == 0: continue
    #         conj = re.sub(", ", "",matches[idx])
    #         parts[idx] = part
    #         parts[idx + 1] = conj + parts[idx+1]
    #     all_parts.extend(parts)
    return_parts = []
    merge = False 
    for part in subsegments:
        part = part.strip(" ")
        # if len(part) > 1 and not re.search(r"^\)\s*$|^\(\s*$|^[bB]ehold[\s\,]*$|^say[\s\,]*|^and say[\s\,]*|^and you$",part): 
        if re.search(r"say|saith",part): 
            # merge with prior part
            if len(return_parts) == 0: 
                merge = True
                return_parts.append(part)
            else: 
                return_parts[-1] = return_parts[-1] + " " + part  
        else: 
            if merge: 
                merge = False 
                return_parts[-1] = return_parts[-1] + " " + part
            else: 
                return_parts.append(part) 
    return return_parts

In [128]:
segmented_bible = {}
for label, verse in bible.items(): 
    segmented_bible[label] = split_verse(verse)
bible_labels = []
bible_parts = [] 
for label, verse in segmented_bible.items(): 
    for part in verse:
        if len(part.split(" ")) < 2: continue
        bible_parts.append(part)
        bible_labels.append(label)

In [129]:
lengths = [len(s.split(" ")) for s in bible_parts]
import numpy as np 
np.percentile(lengths,25),np.percentile(lengths,50),np.percentile(lengths,75),np.percentile(lengths,99)

(15.0, 21.0, 28.0, 52.0)

In [130]:
vector = TfidfVectorizer(norm=None, analyzer='word',sublinear_tf=True)
tfidf_bible = vector.fit_transform(bible_parts)
df = pd.DataFrame(tfidf_bible.toarray(), columns = vector.get_feature_names_out())

In [131]:
doc_scores = df.sum(axis=1)

In [132]:
# get the average TFIDF score for each verse 
for idx, score in enumerate(doc_scores): 
    num_words = len(bible_parts[idx].split(" "))
    doc_scores[idx] = score/num_words
sorted_docs = np.argsort(doc_scores)[::-1]

In [133]:
with open('../assets/bible/bible_parts.txt','w') as file: 
    for idx in sorted_docs: 
        file.write(bible_labels[idx]+"\n")
        file.write(bible_parts[idx]+"\n")

In [134]:
import re
bible = {}
outputfolder = "/Users/amycweng/DH/SERMONS_APP/db/data"
with open(f'../assets/bible/bible_parts.txt','r') as file:
    lines = file.readlines()
    for idx, line in enumerate(lines):
      if len(re.findall("-",line)) >= 2 and not re.search(" ", line.strip("\n")):
        verse = line.strip("\n")
        text = lines[idx+1].strip("\n").strip(" ")
        # if re.search(r"^or else$|^yea$|^moreover$|^[lL]o$|^if otherwise$|^wait$|^and yet true$",text): continue
        if verse not in bible: bible[verse] = []
        bible[verse].append(text)


In [135]:
len(bible)

36716

In [136]:
unique = []
for p_list in bible.values(): 
    unique.extend(p_list)
unique = {phrase:idx for idx, phrase in enumerate(set(unique))}
unique_bible = {idx:[] for idx in unique.values()}
for label, v_list in bible.items():
    for verse in v_list:
        vidx = unique[verse]
        unique_bible[vidx].append(label)
phrases_csv = []
for phrase, vidx in unique.items(): 
    phrases_csv.append({"vidx":vidx, "phrase":phrase})

bible_vindices = []
for label, vlist in bible.items(): 
    seen = []
    label = re.sub("-"," ",label).split(" ")
    label = " ".join(label[:-2]) + " " + label[-2] + ":" + label[-1] + " (KJV)"
    for v in vlist: 
        vidx = unique[v]
        if vidx in seen: continue
        bible_vindices.append({'vidx':vidx, 'verse_id':label})
        seen.append(vidx)

In [137]:
with open(f"{outputfolder}/bible_phrase_indices.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=bible_vindices[0].keys())
    writer.writerows(bible_vindices)
with open(f"{outputfolder}/bible_phrases.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=phrases_csv[0].keys())
    writer.writerows(phrases_csv)